<a href="https://colab.research.google.com/github/mattia-sandrini/deep-learning-book/blob/master/Linear_Regression/linear-regression-gluon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install mxnet
!pip install d2l

     |████████████████████████████████| 68.7MB 60kB/s 
  Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1


In [20]:
import d2l
from mxnet import autograd, nd, gluon, init
from mxnet.gluon import nn, loss as gloss

# Generate the dataset
def synthetic_data(w, b, num_examples):
  X = nd.random.normal(scale=1, shape=(num_examples, len(w)))
  y = nd.dot(X, w) + b
  y += nd.random.normal(scale=0.01, shape=y.shape)
  return X, y

true_w = nd.array([2, -3.4])
true_b = 4.2
features, labels = synthetic_data(true_w, true_b, 1000)


# Define the data loader
def load_array(data_arrays, batch_size, is_train=True):
  """ Construct a Gluon data loader """
  dataset = gluon.data.ArrayDataset(*data_arrays)
  return gluon.data.DataLoader(dataset, batch_size, shuffle=is_train)

batch_size = 10
data_iter = load_array((features, labels), batch_size)

# Define the Model as a sequential Neural Network with a single fully-connected Dense layer
net = nn.Sequential()
net.add(nn.Dense(1))    # Add a fully-connected layer as output of 1 scalar

# Initialize model parameters
net.initialize(init.Normal(sigma=0.01))

# Loss function
loss = gloss.L2Loss()

# Optimization algorithm
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': 0.03})

# Training
num_epochs = 3
for epoch in range(1, num_epochs+1):
  for X, y in data_iter:
    with autograd.record():
      l = loss(net(X), y)
    l.backward()
    trainer.step(batch_size)
  l = loss(net(features), labels)
  print(l.mean().asnumpy())
  print(type(l.mean().asnumpy()))
  print('epoch %d, loss: %f' % (epoch, l.mean().asnumpy()))
  # NB: asnumpy() convert a <NDArray 1 @cpu(0)> into a numpy.ndarray with a single element 

  # Access the learned parameters
  w = net[0].weight.data()
  b = net[0].bias.data()
  print('\n\nError in stimating w', true_w.reshape(w.shape) - w)
  print('\nError in stimating b', true_b - b)

[0.04907026]
<class 'numpy.ndarray'>
epoch 1, loss: 0.049070


Error in stimating w 
[[ 0.16474235 -0.1715529 ]]
<NDArray 1x2 @cpu(0)>

Error in stimating b 
[0.22070003]
<NDArray 1 @cpu(0)>
[0.00023197]
<class 'numpy.ndarray'>
epoch 2, loss: 0.000232


Error in stimating w 
[[ 0.01296377 -0.00927782]]
<NDArray 1x2 @cpu(0)>

Error in stimating b 
[0.01145458]
<NDArray 1 @cpu(0)>
[5.3800635e-05]
<class 'numpy.ndarray'>
epoch 3, loss: 0.000054


Error in stimating w 
[[0.00120294 0.00023365]]
<NDArray 1x2 @cpu(0)>

Error in stimating b 
[-0.00016022]
<NDArray 1 @cpu(0)>
